## 任务1：读取汽车问答数据

In [ ]:
import json
import pdfplumber

questions = json.load(open("../data/Coggle比赛数据/汽车知识问答/questions.json"))
print(questions[0])

pdf = pdfplumber.open("../data/Coggle比赛数据/汽车知识问答/初赛训练数据集.pdf")
pdf.pages[0].extract_text() # 可复制的PDF
pdf.pages[0]

In [ ]:
# 将pdf内容存放在列表里
pdf_content=[]

# 将page页数和内容放在一个字典里
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page':'page_'+str(page_idx+1),
        'content':pdf.pages[page_idx].extract_text()
    })
pdf_content[0]


In [ ]:
questions[:10]

## 任务2：文本索引与答案检索
## TFIDF

In [ ]:
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [ ]:
# 对文本进行分词
question_words=[f''.join(jieba.lcut(x['question'])) for x in questions]
pdf_content_words=[''.join(jieba.lcut(x['content'])) for x in pdf_content]

In [ ]:
# TFIDF倒排序
tfidf=TfidfVectorizer()
tfidf.fit(question_words+pdf_content_words)

question_feat = tfidf.transform(question_words) # 转换成矩阵
pdf_content_feat=tfidf.transform(pdf_content_words) # 转换成矩阵

# 归一化，方便后面相似度的一个打分,得到一个归一化的矩阵
question_feat=normalize(question_feat)
pdf_content_feat=normalize(pdf_content_feat)

In [ ]:
# 通过TFIDF进行检索
for quer_idx,feat in enumerate(question_feat):

    # 对每个提问与每页PDF进行打分
    score = feat @ pdf_content_feat.T
    score = score.toarray()[0]

    max_score_page_idx = score.argsort()[::-1][0]+1
    questions[quer_idx]["reference"] = 'page_'+str(max_score_page_idx)

In [ ]:
# 保存检索完成的数据集
with open('../data/Coggle比赛数据/汽车知识问答/submit_tfidf.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

## BM25

In [ ]:
from rank_bm25 import BM25Okapi

pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

In [ ]:
for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idx = doc_scores.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

In [ ]:
with open('../data/Coggle比赛数据/汽车知识问答/submit_bm25.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

## 任务3：语义检索